<a href="https://colab.research.google.com/github/4may/SuperResolutionSurvey/blob/master/FSRCNN_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fast Super-Resolution Convolutional Network(FSRCNN)

[OSSのソース](https://github.com/GeorgeSeif/FSRCNN-Keras)を参考に実装してみる。

* data
  * trainingデータとして、291種類の画像データを使用している。
    * 明確なデータが確認できなかったが、おそらく91(.bmp)+200(.jpg)で構成されている
    * matlabのプログラムでデータを加工しているのだが、拡張子が.bmpと.jpgの二つある
      * [91](https://www.kaggle.com/ll01dm/t91-image-dataset):.bmpファイル。
      * [200](https://www2.eecs.berkeley.edu/Research/Projects/CS/vision/bsds/BSDS300/html/dataset/images.html):.jpgファイル。確度そこそこ。train/testで分かれていて、それぞれ200と100枚。
  * testデータとして、5種類の画像データを使用している。
    * こちらは、明確にSet5との記載あり。
* アーキテクチャ
  * コスパの良いFSRCNN(56, 16, 4)が採用されている。

## Data Augmentation

[matlabのソース](https://github.com/GeorgeSeif/FSRCNN-Keras/blob/master/data/aug_train.m)を参考に、291種類の画像データに対してdata augmentationを行う。

In [3]:
!unzip "291.zip"

Archive:  291.zip
   creating: 291/
  inflating: 291/157036.jpg          
  inflating: __MACOSX/291/._157036.jpg  
  inflating: 291/187029.jpg          
  inflating: __MACOSX/291/._187029.jpg  
  inflating: 291/71046.jpg           
  inflating: __MACOSX/291/._71046.jpg  
  inflating: 291/299091.jpg          
  inflating: __MACOSX/291/._299091.jpg  
  inflating: 291/tt26.bmp            
  inflating: __MACOSX/291/._tt26.bmp  
  inflating: 291/242078.jpg          
  inflating: __MACOSX/291/._242078.jpg  
  inflating: 291/t53.bmp             
  inflating: __MACOSX/291/._t53.bmp  
  inflating: 291/t47.bmp             
  inflating: __MACOSX/291/._t47.bmp  
  inflating: 291/232038.jpg          
  inflating: __MACOSX/291/._232038.jpg  
  inflating: 291/173036.jpg          
  inflating: __MACOSX/291/._173036.jpg  
  inflating: 291/198054.jpg          
  inflating: __MACOSX/291/._198054.jpg  
  inflating: 291/t46.bmp             
  inflating: __MACOSX/291/._t46.bmp  
  inflating: 291/388016.jpg 

In [0]:
import cv2
import numpy as np
import os
from glob import glob
import math
import matplotlib.pyplot as plt
import imageio
from scipy.io import savemat
from scipy.io import loadmat

#freeimageをdownloadしておかないとエラーとなる。
imageio.plugins.freeimage.download()

291種類のファイルのパスを取得する。

In [0]:
#291種類の画像データが格納されているフォルダ
TRAIN_SOURCE_DIR = '291'

#data augmentationをした画像を保存するフォルダ
TRAIN_TARGET_DIR = 'train'

sources = glob("{}/*.jpg".format(TRAIN_SOURCE_DIR)) + glob("{}/*.bmp".format(TRAIN_SOURCE_DIR))

In [126]:
sources[:5]

['291/202012.jpg',
 '291/254054.jpg',
 '291/176039.jpg',
 '291/106020.jpg',
 '291/372047.jpg']

In [127]:
sources[286:]

['291/t14.bmp', '291/t9.bmp', '291/tt12.bmp', '291/t55.bmp', '291/tt3.bmp']

In [128]:
len(sources)

291

各ファイルパスから得られる画像に対して、data augmentationを行う。

In [209]:
%%time

#保存先ディレクトリの作成
if os.path.isdir(TRAIN_TARGET_DIR) == False:
  os.mkdir(TRAIN_TARGET_DIR)

#パッチの総数
patch_count = 0
#処理した画像の枚数
img_num = 0
for source in sources:
  #パスから画像をロード
  img_raw = cv2.imread(source)
  img_raw = cv2.cvtColor(img_raw, cv2.COLOR_BGR2YCrCb)
  
  #ピクセル値を0.0~1.0の間に正規化
  #https://ja.coder.work/so/python/385751
  img_raw = cv2.normalize(img_raw[:, :, 1].astype(float), None, 0.0, 1.0, cv2.NORM_MINMAX)

  height, width = img_raw.shape
  size = (width, height)

  #クロッピング
  img_raw = img_raw[1:(height-np.mod(height, 12)), 0:(width-np.mod(width, 12))]

  img_shape = img_raw.shape
  patch_size = 32
  stride = 32
  x_size = math.floor((width - patch_size)/stride + 1)
  y_size = math.floor((height - patch_size)/stride + 1)

  #data augmentation
  for x in range(x_size):
    for y in range(y_size):
      x_coord = x * stride
      y_coord = y * stride

      #パッチその１(クリッピングのみ)
      patch_name = os.path.join(TRAIN_TARGET_DIR, str(patch_count))
      patch = img_raw[y_coord+1:y_coord+patch_size, x_coord+1:x_coord+patch_size]
      #imageio.imwrite('{}.exr'.format(patch_name), patch.astype('float32'), format='EXR-FI')
      savemat('{}.mat'.format(patch_name), {'patch':patch})
      ph, pw = patch.shape
      patch_2 = cv2.resize(patch, (ph // 2, pw // 2), cv2.INTER_CUBIC)
      #imageio.imwrite('{}_2.exr'.format(patch_name), patch_2.astype('float32'), format='EXR-FI')
      savemat('{}_2.mat'.format(patch_name), {'patch':patch_2})
      patch_3 = cv2.resize(patch, (ph // 3, pw // 3), cv2.INTER_CUBIC)
      #imageio.imwrite('{}_3.exr'.format(patch_name), patch_3.astype('float32'), format='EXR-FI')
      savemat('{}_3.mat'.format(patch_name), {'patch':patch_3})
      patch_4 = cv2.resize(patch, (ph // 4, pw // 4), cv2.INTER_CUBIC)
      #imageio.imwrite('{}_4.exr'.format(patch_name), patch_4.astype('float32'), format='EXR-FI')
      savemat('{}_4.mat'.format(patch_name), {'patch':patch_4})

      patch_count += 1

      #パッチその2(クリッピング+90°回転)
      patch_name = os.path.join(TRAIN_TARGET_DIR, str(patch_count))
      patch = img_raw[y_coord+1:y_coord+patch_size, x_coord+1:x_coord+patch_size]
      patch = np.rot90(patch, k=1)
      #imageio.imwrite('{}.exr'.format(patch_name), patch.astype('float32'), format='EXR-FI')
      savemat('{}.mat'.format(patch_name), {'patch':patch})
      ph, pw = patch.shape
      patch_2 = cv2.resize(patch, (ph // 2, pw // 2), cv2.INTER_CUBIC)
      #imageio.imwrite('{}_2.exr'.format(patch_name), patch_2.astype('float32'), format='EXR-FI')
      savemat('{}_2.mat'.format(patch_name), {'patch':patch_2})
      patch_3 = cv2.resize(patch, (ph // 3, pw // 3), cv2.INTER_CUBIC)
      #imageio.imwrite('{}_3.exr'.format(patch_name), patch_3.astype('float32'), format='EXR-FI')
      savemat('{}_3.mat'.format(patch_name), {'patch':patch_3})
      patch_4 = cv2.resize(patch, (ph // 4, pw // 4), cv2.INTER_CUBIC)
      #imageio.imwrite('{}_4.exr'.format(patch_name), patch_4.astype('float32'), format='EXR-FI')
      savemat('{}_4.mat'.format(patch_name), {'patch':patch_4})

      patch_count += 1

      #パッチその3(クリッピング+左右反転)
      patch_name = os.path.join(TRAIN_TARGET_DIR, str(patch_count))
      patch = img_raw[y_coord+1:y_coord+patch_size, x_coord+1:x_coord+patch_size]
      patch = np.fliplr(patch)
      #imageio.imwrite('{}.exr'.format(patch_name), patch.astype('float32'), format='EXR-FI')
      savemat('{}.mat'.format(patch_name), {'patch':patch})
      ph, pw = patch.shape
      patch_2 = cv2.resize(patch, (ph // 2, pw // 2), cv2.INTER_CUBIC)
      #imageio.imwrite('{}_2.exr'.format(patch_name), patch_2.astype('float32'), format='EXR-FI')
      savemat('{}_2.mat'.format(patch_name), {'patch':patch_2})
      patch_3 = cv2.resize(patch, (ph // 3, pw // 3), cv2.INTER_CUBIC)
      #imageio.imwrite('{}_3.exr'.format(patch_name), patch_3.astype('float32'), format='EXR-FI')
      savemat('{}_3.mat'.format(patch_name), {'patch':patch_3})
      patch_4 = cv2.resize(patch, (ph // 4, pw // 4), cv2.INTER_CUBIC)
      #imageio.imwrite('{}_4.exr'.format(patch_name), patch_4.astype('float32'), format='EXR-FI')
      savemat('{}_4.mat'.format(patch_name), {'patch':patch_4})

      patch_count += 1

      #パッチその4(クリッピング+左右反転+90°回転)
      patch_name = os.path.join(TRAIN_TARGET_DIR, str(patch_count))
      patch = img_raw[y_coord+1:y_coord+patch_size, x_coord+1:x_coord+patch_size]
      patch = np.fliplr(patch)
      patch = np.rot90(patch, k=1)
      #imageio.imwrite('{}.exr'.format(patch_name), patch.astype('float32'), format='EXR-FI')
      savemat('{}.mat'.format(patch_name), {'patch':patch})
      ph, pw = patch.shape
      patch_2 = cv2.resize(patch, (ph // 2, pw // 2), cv2.INTER_CUBIC)
      #imageio.imwrite('{}_2.exr'.format(patch_name), patch_2.astype('float32'), format='EXR-FI')
      savemat('{}_2.mat'.format(patch_name), {'patch':patch_2})
      patch_3 = cv2.resize(patch, (ph // 3, pw // 3), cv2.INTER_CUBIC)
      #imageio.imwrite('{}_3.exr'.format(patch_name), patch_3.astype('float32'), format='EXR-FI')
      savemat('{}_3.mat'.format(patch_name), {'patch':patch_3})
      patch_4 = cv2.resize(patch, (ph // 4, pw // 4), cv2.INTER_CUBIC)
      #imageio.imwrite('{}_4.exr'.format(patch_name), patch_4.astype('float32'), format='EXR-FI')
      savemat('{}_4.mat'.format(patch_name), {'patch':patch_4})

      patch_count += 1
  
  img_num += 1
  print(img_num)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [210]:
img = cv2.imread('291/2092.jpg')
img.shape

(321, 481, 3)

In [211]:
img2 = cv2.imread('Set5/baby_GT.bmp')
img2.shape

(512, 512, 3)

exrファイルの場合、10分ぐらいかかったが、matファイルの場合は3分弱だった。

data augmentationした画像が保存できているかどうか確認する。

In [212]:
#img = imageio.imread('train/0.exr')
img_array = loadmat('train/0.mat')['patch']
img_array.shape

(31, 31)

In [213]:
#img[0, 0]
img_array[0]

array([0.14583333, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
       0.14583333, 0.14583333, 0.125     , 0.125     , 0.125     ,
       0.125     , 0.125     , 0.125     , 0.14583333, 0.14583333,
       0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.1875    ,
       0.1875    , 0.1875    , 0.1875    , 0.1875    , 0.1875    ,
       0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
       0.16666667])

In [215]:
#type(img[0, 0, 0])
type(img_array[0, 0])

numpy.float64

In [216]:
img_array = loadmat('train/0_4.mat')['patch']
img_array.shape

(7, 7)

In [217]:
img_array[0]

array([0.16241497, 0.14285714, 0.1335034 , 0.16666667, 0.1875    ,
       0.18027211, 0.18154762])

In [219]:
type(img_array[0, 0])

numpy.float64

続いて、testデータもdata augmentationする。testデータのdata augmentationの手順は、traindデータのそれとほぼ一緒。testデータのdata augmentationでは、パッチの種類が１種類である点だけが異なる。

まずは、test画像のファイルパスを取得する。

In [13]:
%%time
!unzip 'Set5.zip'

Archive:  Set5.zip
   creating: Set5/
  inflating: __MACOSX/._Set5         
  inflating: Set5/woman_GT.bmp       
  inflating: __MACOSX/Set5/._woman_GT.bmp  
  inflating: Set5/butterfly_GT.bmp   
  inflating: __MACOSX/Set5/._butterfly_GT.bmp  
  inflating: Set5/bird_GT.bmp        
  inflating: __MACOSX/Set5/._bird_GT.bmp  
  inflating: Set5/head_GT.bmp        
  inflating: __MACOSX/Set5/._head_GT.bmp  
  inflating: Set5/baby_GT.bmp        
  inflating: __MACOSX/Set5/._baby_GT.bmp  


In [0]:
TEST_SOURCE_DIR = 'Set5'
TEST_TARGET_DIR = 'test'

test_sources = glob('{}/*.bmp'.format(TEST_SOURCE_DIR))

In [140]:
len(test_sources)

5

In [141]:
test_sources

['Set5/woman_GT.bmp',
 'Set5/bird_GT.bmp',
 'Set5/head_GT.bmp',
 'Set5/butterfly_GT.bmp',
 'Set5/baby_GT.bmp']

取得した画像ファイルのパスを使って、data augmentationする。

In [222]:
%%time
#data augmentationしたtest画像の保存先ディレクトリを作成
if os.path.isdir(TEST_TARGET_DIR) == False:
  os.mkdir(TEST_TARGET_DIR)

#data augmentation
path_count = 0
img_count = 0

for source in test_sources:
  #YCrCb空間で読み取り
  img_raw = cv2.imread(source)
  img_raw = cv2.cvtColor(img_raw, cv2.COLOR_BGR2YCrCb)
  img_raw = img_raw[:, :, 1]
  
  #正規化
  img_raw = cv2.normalize(img_raw.astype(float), None, 0.0, 1.0, cv2.NORM_MINMAX)

  #クロッピング
  height, width = img_raw.shape
  img_raw = img_raw[1:(height-np.mod(height, 12)), 0:(width-np.mod(width, 12))]

  #パッチ化
  patch_size = 32
  stride = 14
  x_size = math.floor((width - patch_size)/stride) + 1
  y_size = math.floor((height - patch_size)/stride) + 1
  for x in range(x_size):
    for y in range(y_size):
      x_coord = x * stride
      y_coord = y * stride

      patch_name = os.path.join(TEST_TARGET_DIR, str(path_count))
      patch = img_raw[y_coord+1:y_coord+patch_size, x_coord+1:x_coord+patch_size]
      #ピクセル値をfloat32のまま保存しておきたい。そのため、imageio.imwrite関数を使う。
      #ポイントは、拡張子として.exrを指定することと、formatとして'EXR-FI'を指定すること。
      #なお、ファイルを呼び出すときは、imageio.imread('hoge.exr')とすれば良い。
      #また、patchのピクセル値の型はfloat64であるが、float64は保存できないため、float32にする
      #imageio.imwrite('{}.exr'.format(patch_name), patch.astype('float32'), format='EXR-FI')
      savemat('{}.mat'.format(patch_name), {'patch':patch})
      ph, pw = patch.shape
      patch_2 = cv2.resize(patch, (ph // 2, pw // 2), cv2.INTER_CUBIC)
      #imageio.imwrite('{}_2.exr'.format(patch_name), patch_2.astype('float32'), format='EXR-FI')
      savemat('{}_2.mat'.format(patch_name), {'patch':patch_2})
      patch_3 = cv2.resize(patch, (ph // 3, pw // 3), cv2.INTER_CUBIC)
      #imageio.imwrite('{}_3.exr'.format(patch_name), patch_3.astype('float32'), format='EXR-FI')
      savemat('{}_3.mat'.format(patch_name), {'patch':patch_3})
      patch_4 = cv2.resize(patch, (ph // 4, pw // 4), cv2.INTER_CUBIC)
      #imageio.imwrite('{}_4.exr'.format(patch_name), patch_4.astype('float32'), format='EXR-FI')
      savemat('{}_4.mat'.format(patch_name), {'patch':patch_4})

      path_count += 1
  
  img_count += 1
  print(img_count)

1
2
3
4
5
CPU times: user 1.57 s, sys: 774 ms, total: 2.34 s
Wall time: 2.37 s


作成したmatファイルから、画像を読み出してみよう。画像を読み出すためには、scipy.io.loadmat()を使う。

In [0]:
img_dict = loadmat('test/111.mat')

img_dictオブジェクトに、matファイルの中身を読み込めた。

次に、img_dictのkeyを確認してみよう。

In [224]:
img_dict.keys()

dict_keys(['__header__', '__version__', '__globals__', 'patch'])

img_dictのキーには、headerやversionなどのメタ情報の他、自分で定義したpatchというキーがある。

このpatchキーに対応する値として、画像のピクセル値が格納されている。確かめてみよう。

In [0]:
img_array = img_dict['patch']

In [226]:
img_array.shape

(31, 31)

In [227]:
img_array[0]

array([0.09375 , 0.109375, 0.109375, 0.109375, 0.109375, 0.109375,
       0.109375, 0.125   , 0.140625, 0.171875, 0.203125, 0.25    ,
       0.296875, 0.34375 , 0.390625, 0.421875, 0.4375  , 0.453125,
       0.46875 , 0.453125, 0.453125, 0.4375  , 0.4375  , 0.4375  ,
       0.4375  , 0.4375  , 0.4375  , 0.4375  , 0.4375  , 0.421875,
       0.421875])

In [229]:
type(img_array[0, 0])

numpy.float64

float64型の値が、二次元配列として格納されていることがわかる。

念のため、scaleが異なる画像も読み込めているか確認しよう。

In [230]:
#img = imageio.imread('test/0.exr')
img_array = loadmat('test/111_4.mat')['patch']
img_array.shape

(7, 7)

In [231]:
#img[0, 0]
img_array[0, 0]

0.09821428544819355

In [233]:
#type(img[0, 0])
type(img_array[0, 0])

numpy.float64

読み込めてるね！

## Learning

以下の手順で行う。

1. データをロードする
2. 学習モデルを作成する
3. 学習する

In [21]:
import re
from keras.models import Sequential, Model
from keras.layers import Input, Conv2D, Conv2DTranspose
from keras.layers.advanced_activations import PReLU
import tensorflow as tf
from keras import backend as K
import threading

Using TensorFlow backend.


In [0]:
TRAIN_SCALES = [4]
VALID_SCALES = [4]
IMG_SIZE = (32, 32, 1)
INPUT_SCALE = 4
BATCH_SIZE = 128
EPOCHS = 500

**1.データをロードする**

In [0]:
def get_image_list(data_path, scales=[2, 3, 4]):
  #scaleしていない画像パスの一覧を取得
  #ファイルの拡張子は、.matである。
  image_paths = glob(os.path.join(data_path, "*"))
  image_paths = [image_path for image_path in image_paths if re.search("^\d+.mat$", os.path.basename(image_path))]
  
  #スケール済み画像も含めた画像パスの一覧
  train_list = []
  
  #各種スケール済み画像を読み込み
  for image_path in image_paths:
    if os.path.exists(image_path):
      for scale in scales:
        #末尾の4文字(拡張子)は除く
        scale_image_path = image_path[:-4] + "_" + str(scale) + ".mat"
        if os.path.exists(scale_image_path):
          train_list.append([image_path, scale_image_path])
  
  return train_list

In [0]:
#trainingデータを読み込み
train_list = get_image_list('train', scales=TRAIN_SCALES)

In [188]:
len(train_list)

138100

In [189]:
train_list[:5]

[['train/70477.mat', 'train/70477_4.mat'],
 ['train/34080.mat', 'train/34080_4.mat'],
 ['train/90828.mat', 'train/90828_4.mat'],
 ['train/79685.mat', 'train/79685_4.mat'],
 ['train/137277.mat', 'train/137277_4.mat']]

In [0]:
#validationデータを読み込み
test_list = get_image_list('test', scales=VALID_SCALES)

In [191]:
len(test_list)

2544

In [192]:
test_list[:5]

[['test/2093.mat', 'test/2093_4.mat'],
 ['test/1861.mat', 'test/1861_4.mat'],
 ['test/1839.mat', 'test/1839_4.mat'],
 ['test/312.mat', 'test/312_4.mat'],
 ['test/268.mat', 'test/268_4.mat']]

**2.学習モデルの作成**

FSRCNN(56, 16, 4)を実装する。

まず、PSNRを定義する。

In [0]:
def tf_log10(x):
  numerator = tf.math.log(x)
  denominator = tf.math.log(tf.constant(10, dtype=numerator.dtype))
  return numerator / denominator

In [0]:
def PSNR(y_true, y_pred):
  #データ作成時にnp.normalize()で正規化済み
  max_pixel = 1.0
  print(psnr)
  return 10.0 * tf_log10((max_pixel ** 2) / (K.mean(K.square(y_pred - y_true))))

In [0]:
y_true = np.arange(0, 10, dtype=np.float32)
y_pred = np.arange(1, 11, dtype=np.float32)

In [55]:
y_true

array([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.], dtype=float32)

In [56]:
y_pred

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.], dtype=float32)

In [57]:
psnr = PSNR(y_true, y_pred)
psnr

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

次に、学習モデルを作成する。

In [68]:
input_img = Input(shape=(IMG_SIZE[0]//INPUT_SCALE, IMG_SIZE[1]//INPUT_SCALE, 1))

#feature extraction layer
model = Conv2D(56, (5, 5), padding='same', kernel_initializer='he_normal')(input_img)
model = PReLU()(model)

#shrinking layer
model = Conv2D(16, (1, 1), padding='same', kernel_initializer='he_normal')(model)
model = PReLU()(model)

#non-linear mapping layer
model = Conv2D(12, (3, 3), padding='same', kernel_initializer='he_normal')(model)
model = PReLU()(model)
model = Conv2D(12, (3, 3), padding='same', kernel_initializer='he_normal')(model)
model = PReLU()(model)
model = Conv2D(12, (3, 3), padding='same', kernel_initializer='he_normal')(model)
model = PReLU()(model)
model = Conv2D(12, (3, 3), padding='same', kernel_initializer='he_normal')(model)
model = PReLU()(model)

#
model = Conv2D(56, (1, 1), padding='same', kernel_initializer='he_normal')(model)
model = PReLU()(model)

model = Conv2DTranspose(1, (9, 9), strides=(4, 4), padding='same')(model)

output_img = model

model = Model(input_img, output_img)

model.compile(optimizer='adam', loss='mse', metrics=[PSNR, "accuracy"])

model.summary()

tf.Tensor(0.0, shape=(), dtype=float32)
Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 8, 8, 1)           0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 8, 8, 56)          1456      
_________________________________________________________________
p_re_lu_15 (PReLU)           (None, 8, 8, 56)          3584      
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 8, 8, 16)          912       
_________________________________________________________________
p_re_lu_16 (PReLU)           (None, 8, 8, 16)          1024      
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 8, 8, 12)          1740      
_________________________________________________________________
p_re_lu_17 (PReLU) 

**3.学習する**

学習データのサイズが大きいので、iteratorを使うことにする。ここでは、新たに二つの関数を定義する。

* get_image_batch : 引数のリストから二つの部分データ(バッチ)を生成する。
* image_gen : genarator。get_image_batchの戻り値二つをiteratorとしてまとめて返す。

In [0]:
def get_image_batch(train_list, offset, scale):
  '''
  train_listのoffset番目の要素から始めて、BATCH_SIZE個分の画像データを取得する。
  スケール前の画像とスケール後の画像の二つを分けて取得する。

  Parameters
  ---------------
  train_list : 学習対象のデータ
  offset : データを読み始める位置。train_list内の位置を示す。
  scale : 画像スケール

  Returns
  ----------
  input_list : スケール後の画像の一覧。要素数はBATCH_SIZE
  gt_list : スケール前の画像の一覧。要素数はBATCH_SIZE
  '''
  #train_listからBATCH_SIZE個の要素を取得する
  #個々の要素は、以下のようにオリジナルの画像パスとscale後の画像パスのペアとなっている。
  #['train/70477.mat', 'train/70477_4.mat']
  target_list = train_list[offset:offset+BATCH_SIZE]

  #入力画像(スケール済み)データの一覧
  input_list = []
  #Ground Truth画像(スケールなし)データの一覧
  gt_list = []
  #各画像データの取得
  for pair in target_list:
    input_img = loadmat(pair[1])['patch']
    gt_img = loadmat(pair[0])['patch']
    input_list.append(input_img)
    gt_list.append(gt_img)
  
  #データ整形
  input_list = np.array(input_list)
  input_list.resize([BATCH_SIZE, IMG_SIZE[0]//scale, IMG_SIZE[1]//scale, 1])

  gt_list = np.array(gt_list)
  gt_list.resize([BATCH_SIZE, IMG_SIZE[0], IMG_SIZE[1], 1])

  return input_list, gt_list

In [0]:
def image_gen(target_list, scale):
  '''
  2種類の学習データを返すgenerator

  Parameters
  ---------------
  target_list : 画像データの一覧
  scale : 画像スケール(元画像をどれだけ小さくするか. 2, 3, 4のいずれかの値を取る。)

  Returns
  ----------
  (batch_x, batch_y) : スケール前画像とスケール後画像のタプル。iterator。
  '''
  length = len(target_list)
  #ステップ数(バッチの数)
  num_steps = length // BATCH_SIZE
  
  #generator
  while True:
    for step in range(num_steps):
      offset = step*BATCH_SIZE
      batch_x, batch_y = get_image_batch(target_list, offset, scale)
      yield (batch_x, batch_y)

動作確認しておこう。

In [0]:
tmp_list = train_list[:1000]
tmp_iter = image_gen(tmp_list, scale=4)

In [0]:
batch_x, batch_y = next(tmp_iter)

In [236]:
batch_x.shape

(128, 8, 8, 1)

In [237]:
batch_y.shape

(128, 32, 32, 1)

上記の関数により生成されるiteratorを使って、学習する。

各関数は正常に動いている。

それでは、学習してみよう。

In [245]:
model.fit_generator(image_gen(train_list, scale=INPUT_SCALE), 
                    steps_per_epoch=len(train_list) // BATCH_SIZE,
                    validation_data=image_gen(test_list, scale=INPUT_SCALE), 
                    validation_steps = len(train_list) // BATCH_SIZE,
                    epochs=EPOCHS)

Epoch 1/500


ValueError: ignored

In [0]:
def get_batches(target_list, scale):
  '''
  2種類の学習データを返すgenerator

  Parameters
  ---------------
  target_list : 画像データの一覧
  scale : 画像スケール(元画像をどれだけ小さくするか. 2, 3, 4のいずれかの値を取る。)

  Returns
  ----------
  (batch_x, batch_y) : スケール前画像とスケール後画像のタプル。iterator。
  '''
  length = len(target_list)
  #ステップ数(バッチの数)
  num_steps = length // BATCH_SIZE
  
  batches = []
  for step in range(num_steps):
    offset = step*BATCH_SIZE
    batch_x, batch_y = get_image_batch(target_list, offset, scale)
    batches.append((batch_x, batch_y))
  
  return np.array(batches)

In [204]:
batches = get_batches(train_list, scale=4)

ValueError: ignored

以下のようなエラーが発生している。

```
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
<ipython-input-85-d4e4b9f0e01b> in <module>()
----> 1 batches = get_batches(train_list, scale=4)

1 frames
<ipython-input-84-1c42aecbb5fe> in get_batches(target_list, scale)
     20   for step in range(num_steps):
     21     offset = step*BATCH_SIZE
---> 22     batch_x, batch_y = get_image_batch(target_list, offset, scale)
     23     batches.append((batch_x, batch_y))
     24 

<ipython-input-81-28ae2278b734> in get_image_batch(train_list, offset, scale)
     33 
     34   #データ整形
---> 35   input_list = np.array(input_list)
     36   print("before:", input_list.shape)
     37   input_list.resize([BATCH_SIZE, IMG_SIZE[0]//scale, IMG_SIZE[1]//scale, 1])

ValueError: could not broadcast input array from shape (7,7,3) into shape (7)
```

ここで、(7, 7, 3)というshapeをしているのは、imreadで読み取ったスケール済み画像(input_img)。
なお、端の場合、(7, 5, 3)のように少し小さい値をとる。

ちなみに、ground truth画像(gt_img)のshapeは(31, 31, 3)。

以下のような警告が出て学習が止まっている。

```
/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:718: UserWarning: An input could not be retrieved. It could be because a worker has died.We do not have any information on the lost sample.
  UserWarning)
```

[同じ症状の人がいた](https://qiita.com/mosamosa/items/9907a56cab7ae96d76c7)。

ただ、僕の場合はgoogle colabのcontent配下に既にデータを置いている。[いろいろ調べてみると](https://stackoverflow.com/questions/60838253/userwarning-the-input-94-could-not-be-retrieved-it-could-be-because-a-worker-h)、google colab周りが怪しそう・・・

batch数を半分してみたがだめ。

workerの数は１(default)としているしマルチスレッディングが原因ではなさそうだが、一応、OSS通り、generatorをthread-safeにしてみよう。

In [0]:
class threadsafe_iter:
  '''threadsafeにデータを取得するためのiterator'''
  #iteratorを実装するには、__iter__と、__next__を実装すれば良い。
  #なお、threadsafeとするため、mutexを保持するようにしている。
  def __init__(self, it):
    self.it = it
    self.lock = threading.Lock()

  def __iter__(self):
    return self
  
  def __next__(self):
    #mutexを取得する。
    with self.lock:
      return self.it.__next__()

In [0]:
def threadsafe_generator(f):
  '''引数のgenerator fをthread-safeとするdecorator'''
  def g(*a, **kw):
    return threadsafe_iter(f(*a, **kw))
  
  return g

In [0]:
@threadsafe_generator
def image_gen(target_list, scale):
  '''
  2種類の学習データを返すgenerator

  Parameters
  ---------------
  target_list : 画像データの一覧
  scale : 画像スケール(元画像をどれだけ小さくするか. 2, 3, 4のいずれかの値を取る。)

  Returns
  ----------
  (batch_x, batch_y) : スケール前画像とスケール後画像のタプル。iterator。
  '''

  length = len(target_list)
  #ステップ数(バッチの数)
  num_steps = length // BATCH_SIZE
  
  #generator
  while True:
    for step in range(num_steps):
      offset = step*BATCH_SIZE
      batch_x, batch_y = get_image_batch(target_list, offset, scale)
      yield (batch_x, batch_y)

In [1]:
%%time
model.fit_generator(image_gen(train_list, scale=INPUT_SCALE), 
                    steps_per_epoch=len(train_list) // BATCH_SIZE,
                    validation_data=image_gen(test_list, scale=INPUT_SCALE), 
                    validation_steps = len(train_list) // BATCH_SIZE,
                    epochs=EPOCHS)

NameError: ignored

ですよねー。

[ここ](https://stackoverflow.com/questions/58446290/userwarning-an-input-could-not-be-retrieved-it-could-be-because-a-worker-has/58546878#58546878)を読むに、学習中にデータを読み込もうとしてしまっているみたい。

エラーを出力している箇所は以下の通り。queueからデータを取得するところでタイムアウトが起きている。

```
try:
  while self.is_running():
    try:
      future = self.queue.get(block=True)
      inputs = future.get(timeout=30)
      self.queue.task_done()
    except mp.TimeoutError:
      warnings.warn(
        'An input could not be retrieved.'
        ' It could be because a worker has died.'
        'We do not have any information on the lost sample.',
        UserWarning)
      continue
```

[ここ](https://stackoverflow.com/questions/58446290/userwarning-an-input-could-not-be-retrieved-it-could-be-because-a-worker-has/62078064#62078064)を読むには、GPUによる学習に時間を食っている間にCPUが次のデータを用意しようとするのが原因とされている。解決策を二つ挙げている。一つは、1batchごとにかかる学習時間を短くするためにbatchサイズを小さくすること、もう一つはCPUの設定を見直すこと。

In [0]:
model.save('fsrcnn_model.h5')